In [ ]:
import os
import cudf
import time

If LOCAL_SCRATCH_DIR environment variable is not set, this notebook accesses the ../data directory for temporary files.

In [ ]:
DATA_DIR = os.getenv("LOCAL_SCRATCH_DIR", default="../data")
filename = os.path.join(DATA_DIR, "gene_info.tsv")
print("Filename:", filename)

In [ ]:
start = time.time()

In [ ]:
column_names = ["GeneID", "Symbol", "Synonyms", "description", "type_of_gene", "#tax_id"]

genes = cudf.read_csv(filename, usecols=column_names, dtype=str, sep="\t")
gnese = genes.rename(columns={"#tax_id": "tax_id"})

In [ ]:
# cudf does not support query by string
genes = genes[genes["type_of_gene"] == 'protein-coding']

In [ ]:
groups = genes.groupby("tax_id").size().reset_index()

In [ ]:
groups.columns = ["tax_id", "count"]

In [ ]:
groups = groups.sort_values("count", ascending=False)

### Number of human protein-coding genes (tax_id = 9606)

In [ ]:
groups[groups["tax_id"]  == "9606"]

### Top 5 organisms with the most protein-coding genes

In [ ]:
groups.head()

In [ ]:
end = time.time()

In [ ]:
print(f"cuDF: {end - start} sec.")